# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import csv

# dataset links
# twitch-gamers -> https://snap.stanford.edu/data/twitch_gamers.html

def load_data(filename: str) -> list[frozenset[int]]:
    """Load data from file and return it as a list of frozensets."""
    with open(f'data/{filename}/{filename}_edges.csv', newline='') as csvfile:
        edges_reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        edges = list(frozenset((int(a), int(b))) for a,b in list(edges_reader)[1:])
    return edges

graph_name = 'large_twitch'
edges = load_data(graph_name)

In [ ]:
class Trièst():
    def __init__(self, M):
        self.M = M

    def reservoir_sampling(self, t: int) -> int:
        '''Returns -1 if the new edge is not kept in memory, otherwise the index of the reservoir that has to be substituted'''
        if t <= self.M: 
            return t
        else:
            u = np.random.uniform()
            if u < self.M / t:
                return np.random.choice(M)
            else:
                return -1


M = 7
triest = Trièst(M)
x = triest.reservoir_sampling(0)